In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

VARIABLES = ['Age', 'EtCO2', 'PTT', 'BUN', 'Lactate', 'Temp', 'Hgb',
       'HCO3', 'BaseExcess', 'RRate', 'Fibrinogen', 'Phosphate', 'WBC',
       'Creatinine', 'PaCO2', 'AST', 'FiO2', 'Platelets', 'SaO2', 'Glucose',
       'ABPm', 'Magnesium', 'Potassium', 'ABPd', 'Calcium', 'Alkalinephos',
       'SpO2', 'Bilirubin_direct', 'Chloride', 'Hct', 'Heartrate',
       'Bilirubin_total', 'TroponinI', 'ABPs', 'pH']

FEAT_DESCS = []

In [12]:
def get_patient_data(X, pids):
    return X.loc[X['pid'].isin(pids)]

def impute_variable(X_pid, var, strategy, stats):
    '''imputes time series of variable var according to given imputation strategy
    '''
    if X_pid[var].count == 0:
        pass
    return []

In [8]:
X_features = pd.read_csv('train_features.csv')
all_pids = X_features['pid'].drop_duplicates().to_numpy()
train_pids, val_pids = train_test_split(all_pids, random_state=42)

train_stats = get_patient_data(X_features, train_pids).describe()
test_stats = get_patient_data(X_features, test_stats).describe()

X_all = []

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
count,170952.000000,170952.000000,170952.000000,7540.000000,7668.000000,15081.000000,8027.000000,60996.000000,16681.000000,9402.000000,...,4292.000000,146362.000000,535.000000,10433.000000,20432.000000,150114.000000,4004.000000,2861.000000,143641.000000,18807.000000
mean,15768.154850,7.004703,62.061000,33.279708,40.204434,23.190902,2.839130,36.853695,10.629165,23.510886,...,97.969129,97.660383,1.318972,106.235551,31.305190,84.495237,1.625612,7.395152,122.427517,7.367672
std,9186.380803,4.482653,16.431033,7.759864,26.352134,19.897030,2.374801,0.881488,2.072501,4.434302,...,130.503420,2.801402,2.778550,5.956256,5.772644,17.647456,3.195146,25.902217,23.225179,0.074975
min,2.000000,1.000000,15.000000,10.000000,12.500000,1.000000,0.200000,21.000000,3.700000,0.000000,...,12.000000,20.000000,0.010000,70.000000,9.400000,23.000000,0.100000,0.010000,22.000000,6.820000
25%,7824.000000,4.000000,52.000000,29.000000,27.800000,12.000000,1.400000,36.000000,9.200000,21.000000,...,53.000000,96.000000,0.100000,103.000000,27.300000,72.000000,0.500000,0.030000,106.000000,7.330000
50%,15692.500000,7.000000,64.000000,33.500000,32.200000,17.000000,2.100000,37.000000,10.500000,23.900000,...,72.000000,98.000000,0.300000,107.000000,30.900000,83.000000,0.800000,0.160000,119.000000,7.370000
75%,23783.000000,10.000000,74.000000,38.000000,40.500000,27.000000,3.400000,37.000000,12.000000,26.000000,...,105.000000,100.000000,1.130000,110.000000,35.000000,95.000000,1.400000,2.200000,137.000000,7.410000
max,31658.000000,315.000000,100.000000,100.000000,250.000000,266.000000,31.000000,42.000000,23.800000,50.000000,...,3833.000000,100.000000,21.200000,141.000000,63.400000,191.000000,46.500000,440.000000,287.000000,7.780000


In [11]:
for pid in train_pids:
    X_pid = X_features.loc[X_features['pid'] == pid]

    imputed_time_series={}
    imp_strategy = 'linear_interpolation'
    for var in VARIABLES:
        imputed_time_series[var, imp_strategy] = impute_variable(X_pid, var, imp_strategy, train_stats)

    feat_row = []
    for feat_desc in FEAT_DESCS:
        needed_inputs, fun = feat_desc
        args = []
        for var, needs_imputed, strategy in needed_inputs:
            if needs_inputed:
                input = imputed_time_series[var, strategy]
            else:
                input = X_pid[var].to_numpy()
            args.append(input)
        
        feature = fun(args)
        feat_row.append(feature)
    
    X_all.append(feat_row)

X_train = np.concatenate(X_all, axis=0)

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [int64]